# pyspark

In [ ]:
from glue import *

: 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *
import datetime as dt
import pprint
import boto3
import yaml
pp = pprint.PrettyPrinter(indent = 1)
%load_ext sparksql_magic

creds = read_files()
# subreddit_list = config["subreddit"]
# kafka_host = config["kafka_host"]
# spark_host = config["spark_host"]
kafka_host = "xanaxprincess.asuscomm.com"
spark_host = "xanaxprincess.asuscomm.com"
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]
# subreddit_list = config["subreddit"]
subreddit_list = ["AsiansGoneWild", "GoneWild18", "GoneWild", "GoneWildCurvy", "IndiansGoneWild", "asstastic", "latinas"]
n = len(subreddit_list)
print("{} cores.".format(n))

# subreddit = subreddit_list[0]
for index, subreddit in enumerate(subreddit_list):
    print("subreddit: {}, index: {}".format(subreddit, index))

print("working with subreddit: {}".format(subreddit))

ModuleNotFoundError: No module named 'boto3'

## Spark

In [7]:
subreddit = subreddit_list[1]
os.environ["subreddit"] = subreddit
print(subreddit)

GoneWild18


In [8]:
spark = SparkSession.builder.appName("reddit_" + subreddit + "_glue_partition") \
                    .master("spark://{}:7077".format(spark_host)) \
                    .config("spark.scheduler.mode", "FAIR") \
                    .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                    .config("spark.executor.memory", "1024m") \
                    .config("spark.executor.cores", "2") \
                    .config("spark.streaming.concurrentJobs", "4") \
                    .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                    .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                    .config("spark.sql.debug.maxToStringFields", 1000) \
                    .config("spark.eventLog.enabled", "true") \
                    .config("spark.eventLog.dir", "file:///opt/workspace/events") \
                    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
                    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                    .enableHiveSupport() \
                    .getOrCreate()

print("created spark successfully.")

2022-06-30 23:31:50,596 WARN util.Utils: Your hostname, stevens-mb.local resolves to a loopback address: 127.0.0.1; using 172.16.100.37 instead (on interface en0)
2022-06-30 23:31:50,598 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/stevenhurwitt/miniconda3/envs/reddit/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/stevenhurwitt/.ivy2/cache
The jars for the packages stored in: /Users/stevenhurwitt/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-69c59bd8-8b9d-4e98-b867-3aac08d4b0a5;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in local-m2-cache
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in local-m2-cache
	found org.apache.kafka#kafka-clients;2.8.0 in local-m2-cache
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in local-m2-cache
	found org.spark-proj

RuntimeError: Java gateway process exited before sending its port number

In [9]:
client = boto3.client("s3")

NameError: name 'boto3' is not defined

In [2]:
df = spark.read.format("delta").option("header", True).load("s3a://reddit-stevenhurwitt/" + subreddit)

# df.createOrReplaceTempView("reddit_{}".format(subreddit))
print("created df.")

22/06/26 02:55:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


created df.


In [8]:
df.toPandas()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,NaN,technology,,t2_dvcsbk6c,False,None,0,False,US TikTok user data accessed in China despite ...,r/technology,...,None,/r/technology/comments/veshmq/us_tiktok_user_d...,all_ads,False,https://www.washingtonexaminer.com/policy/tech...,12220571,1.655507e+09,0,None,False
1,NaN,technology,,t2_dvcsbk6c,False,None,0,False,US TikTok user data accessed in China despite ...,r/technology,...,None,/r/technology/comments/veshmq/us_tiktok_user_d...,all_ads,False,https://www.washingtonexaminer.com/policy/tech...,12220573,1.655507e+09,0,None,False
2,NaN,technology,,t2_88icg8jq,False,None,0,False,"Chicago expands and activates quantum network,...",r/technology,...,None,/r/technology/comments/vflnlz/chicago_expands_...,all_ads,False,https://news.uchicago.edu/story/chicago-quantu...,12228022,1.655607e+09,0,None,False
3,NaN,technology,,t2_88icg8jq,False,None,0,False,"Chicago expands and activates quantum network,...",r/technology,...,None,/r/technology/comments/vflnlz/chicago_expands_...,all_ads,False,https://news.uchicago.edu/story/chicago-quantu...,12228022,1.655607e+09,0,None,False
4,NaN,technology,,t2_88icg8jq,False,None,0,False,"Meta is launching an avatar store, and designe...",r/technology,...,None,/r/technology/comments/veuc8b/meta_is_launchin...,all_ads,False,https://www.theverge.com/2022/6/17/23173128/me...,12220972,1.655513e+09,0,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,NaN,technology,,t2_3ed191a2,False,None,0,False,thunderbolt 2 connect issues,r/technology,...,None,/r/technology/comments/vfic7u/thunderbolt_2_co...,all_ads,False,/r/Thunderbolt/comments/vfi7jj/thunderbolt_2_c...,12226938,1.655596e+09,0,None,False
534,NaN,technology,,t2_f6nbn,False,None,0,False,All modern CPUs are vulnerable: dynamic freque...,r/technology,...,None,/r/technology/comments/vcusy0/all_modern_cpus_...,all_ads,False,https://www.hertzbleed.com/,12204618,1.655300e+09,0,None,False
535,NaN,technology,,t2_heajeq9t,False,None,0,False,Looks like Orkut is being resurrected from its...,r/technology,...,None,/r/technology/comments/vggqxw/looks_like_orkut...,all_ads,False,http://www.orkut.com/index.html,12235575,1.655714e+09,0,None,False
536,NaN,technology,,t2_8ytw6jvm,False,None,0,False,"To Build Dark Patterns, I Refuse",r/technology,...,None,/r/technology/comments/vee7w1/to_build_dark_pa...,all_ads,False,https://wagslane.dev/posts/dark-patterns/,12217953,1.655475e+09,0,None,False


In [3]:
df_clean = df.withColumn("approved_at_utc", col("approved_at_utc").cast("timestamp")) \
        .withColumn("banned_at_utc", col("banned_at_utc").cast("timestamp")) \
        .withColumn("created_utc", col("created_utc").cast("timestamp")) \
        .withColumn("created", col("created").cast("timestamp")) \
        .withColumn("date", to_date(col("created_utc"), "MM-dd-yyyy")) \
        .withColumn("year", year(col("date"))) \
        .withColumn("month", month(col("date"))) \
        .withColumn("day", dayofmonth(col("date"))) \
        .dropDuplicates(subset = ["title"])

In [4]:
df_clean_pandas = df_clean.toPandas()
df_clean_pandas.head()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,date,year,month,day
0,NaT,technology,,t2_gfv52,False,None,0,False,"""Traffic Correlation Attacks"" - How oppressive...",r/technology,...,https://windscribbles.com/combating,11996702,2022-05-16 19:05:36,1,None,False,2022-05-16,2022,5,16
1,NaT,technology,,t2_2uwit82z,False,None,0,False,13-Inch MacBook Pro With M2 Chip Outperforms B...,r/technology,...,https://www.macrumors.com/2022/06/16/m2-chip-o...,12211954,2022-06-16 15:49:20,0,None,False,2022-06-16,2022,6,16
2,NaT,technology,,t2_irlcj,False,None,0,False,2023 could be the year Apple switches the iPho...,r/technology,...,https://arstechnica.com/gadgets/2022/05/report...,11974416,2022-05-13 18:33:36,0,None,False,2022-05-13,2022,5,13
3,NaT,technology,,t2_3dv87cht,False,None,0,False,30 year Perovskite solar cell announcement fro...,r/technology,...,https://techxplore.com/news/2022-06-year-perov...,12217816,2022-06-17 14:02:40,0,None,False,2022-06-17,2022,6,17
4,NaT,technology,,t2_guf36,False,None,0,False,52% Of Global Car Buyers Are Now Interested In...,r/technology,...,https://www.19fortyfive.com/2022/06/52-of-glob...,12207039,2022-06-15 21:15:44,0,None,False,2022-06-15,2022,6,15


In [13]:
df_clean.filter(col("title").contains("TikTok")).toPandas()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
0,NaT,technology,,t2_12kvg8j5,False,None,0,False,Elon Musk tells employees he wants Twitter to ...,r/technology,...,/r/technology/comments/vdvkol/elon_musk_tells_...,all_ads,False,https://www.theverge.com/2022/6/16/23171054/el...,12213590,2022-06-16 20:48:00,0,None,False,2022-06-16
1,NaT,technology,,t2_e2flgk7y,False,None,0,False,"Facebook, Twitter, TikTok, Google and others a...",r/technology,...,/r/technology/comments/vfnyqi/facebook_twitter...,all_ads,False,https://www.theverge.com/2022/6/16/23168987/eu...,12228224,2022-06-19 05:17:52,0,None,False,2022-06-19
2,NaT,technology,,t2_c92d05jm,False,None,0,False,"Google Revamps Search, Maps Features for Young...",r/technology,...,/r/technology/comments/uo08gz/google_revamps_s...,all_ads,False,https://www.businessinsider.com/google-revamps...,11964987,2022-05-12 12:45:52,0,None,False,2022-05-12
3,NaT,technology,,t2_88icg8jq,False,None,0,False,Instagram is chasing TikTok with a new full-sc...,r/technology,...,/r/technology/comments/vdsj8g/instagram_is_cha...,all_ads,False,https://www.theverge.com/2022/6/16/23170210/in...,12212842,2022-06-16 18:25:04,0,None,False,2022-06-16
4,NaT,technology,,t2_hr1c9jle,False,None,0,False,Leaked Audio From 80 Internal TikTok Meetings ...,r/technology,...,/r/technology/comments/vel4hw/leaked_audio_fro...,all_ads,False,https://www.buzzfeednews.com/article/emilybake...,12218852,2022-06-17 17:18:56,0,None,False,2022-06-17
5,NaT,technology,,t2_1argepoh,False,None,0,False,Mother Alleges TikTok Challenge Led to Daughte...,r/technology,...,/r/technology/comments/uoxkdc/mother_alleges_t...,all_ads,False,https://gizmodo.com/tiktok-blackout-challenge-...,11974225,2022-05-13 18:03:44,0,None,False,2022-05-13
6,NaT,technology,,t2_88icg8jq,False,None,0,False,TikTok exec: We’re not a social network like F...,r/technology,...,/r/technology/comments/vdwb4t/tiktok_exec_were...,all_ads,False,https://www.cnbc.com/2022/06/16/tiktok-were-an...,12213757,2022-06-16 21:20:00,0,None,False,2022-06-16
7,NaT,technology,,t2_g76xe,False,None,0,False,TikTok moves to ease fears amid report workers...,r/technology,...,/r/technology/comments/vevkm3/tiktok_moves_to_...,all_ads,False,https://www.theguardian.com/technology/2022/ju...,12221238,2022-06-18 01:46:40,0,None,False,2022-06-18
8,NaT,technology,,t2_dvcsbk6c,False,None,0,False,US TikTok user data accessed in China despite ...,r/technology,...,/r/technology/comments/veshmq/us_tiktok_user_d...,all_ads,False,https://www.washingtonexaminer.com/policy/tech...,12220651,2022-06-17 23:00:16,0,None,False,2022-06-17


22/06/20 20:48:05 ERROR TaskSchedulerImpl: Lost executor 4 on 172.26.0.6: worker lost
22/06/20 20:48:05 ERROR TaskSchedulerImpl: Lost executor 3 on 172.26.0.6: worker lost
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_3 !
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_47 !
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_26 !
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_29 !
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_34 !
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_1 !
22/06/20 20:48:05 ERROR TaskSchedulerImpl: Lost executor 5 on 172.26.0.7: worker lost
22/06/20 20:48:05 ERROR TaskSchedulerImpl: Lost executor 0 on 172.26.0.7: worker lost
22/06/20 20:48:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_37 !


In [7]:
filepath = "file:///opt/workspace/technology/"
df.write.format("delta").partitionBy("post_date").mode("overwrite").option("header", True).save(filepath)
print("wrote delta table to local.")

wrote delta table to local.


In [10]:
test = spark.read.format("delta").option("header", True).load(filepath)
test.toPandas()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
0,NaT,technology,,t2_2uwit82z,False,None,0,False,Senator Presses Amazon to Disclose Just How Cr...,r/technology,...,/r/technology/comments/vcdqr2/senator_presses_...,all_ads,False,https://gizmodo.com/amazon-ring-camera-audio-r...,12200483,2022-06-14 21:11:28,0,None,False,2022-06-14
1,NaT,technology,,t2_2uwit82z,False,None,0,False,Ford halts deliveries of electric Mustang Mach...,r/technology,...,/r/technology/comments/vcd10m/ford_halts_deliv...,all_ads,False,https://www.businessinsider.com/ford-mustang-m...,12200136,2022-06-14 20:39:28,0,None,False,2022-06-14
2,NaT,technology,,t2_7i1vm0jb,False,None,0,False,Amazon kicked a trans employee off the board o...,r/technology,...,/r/technology/comments/vcfivu/amazon_kicked_a_...,all_ads,False,https://www.businessinsider.com/amazon-trans-b...,12200666,2022-06-14 22:38:56,0,None,False,2022-06-14
3,NaT,technology,,t2_mr4nwrpm,False,None,0,False,"Wickr, Amazon’s encrypted chat app, has a chil...",r/technology,...,/r/technology/comments/vc63ul/wickr_amazons_en...,all_ads,False,https://www.nbcnews.com/tech/tech-news/wickr-a...,12198321,2022-06-14 15:30:08,0,None,False,2022-06-14
4,NaT,technology,,t2_1ied4ho,False,None,0,False,China’s Chipmaking Power Grows Despite US Effo...,r/technology,...,/r/technology/comments/vc72ia/chinas_chipmakin...,all_ads,False,https://www.bloomberg.com/news/articles/2022-0...,12198605,2022-06-14 16:14:56,0,None,False,2022-06-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,NaT,technology,,t2_8ytw6jvm,False,None,0,False,"To Build Dark Patterns, I Refuse",r/technology,...,/r/technology/comments/vee7w1/to_build_dark_pa...,all_ads,False,https://wagslane.dev/posts/dark-patterns/,12217893,2022-06-17 14:17:36,0,None,False,2022-06-17
247,NaT,technology,,t2_1argepoh,False,None,0,False,Apple iPod creator warns the metaverse will en...,r/technology,...,/r/technology/comments/uo6vg8/apple_ipod_creat...,all_ads,False,https://www.bbc.com/news/business-61423268,11966776,2022-05-12 17:53:04,0,None,False,2022-05-12
248,NaT,technology,,t2_a5jwjm1s,False,None,0,False,A colony of blue-green algae can power a compu...,r/technology,...,/r/technology/comments/uo75rs/a_colony_of_blue...,all_ads,False,https://interestingengineering.com/blue-green-...,11966852,2022-05-12 18:05:52,0,None,False,2022-05-12
249,NaT,technology,,t2_bf38q9nm,False,None,0,False,"Musk, Twitter CEO spar over bot accounts",r/technology,...,/r/technology/comments/ur2j9k/musk_twitter_ceo...,all_ads,False,https://thehill.com/policy/technology/,11996262,2022-05-16 18:27:12,0,None,False,2022-05-16


In [5]:
spark.stop()